Creating an etl Job

In [ ]:
from spark_session_factory import create_spark_session
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window


In [92]:
def run_etl(spark: SparkSession, input_path_topic1: str, input_path_topic2: str, output_path: str):
    """
    Main ETL pipeline: read -> transform -> write.
    
    Args:
        spark: Active SparkSession
        input_path: Landing zone path (e.g., '/opt/spark-data/landing/*.json')
        output_path: Gold zone path (e.g., '/opt/spark-data/gold')
    """
    # TODO: Implement
    spark = spark
    
    df_topic1 = spark.read.json(input_path_topic1).cache()
    df_topic2 = spark.read.json(input_path_topic2).cache()
    
    print(df_topic1.select("*").filter(F.col("user_id") == '10f1bc81').count())
    print(df_topic2.select('*').filter(F.col("user_id") == '10f1bc81').count())
    
    # After a inner join there is an explosion of data copies made
    topics_combined = df_topic1.join(df_topic2,"user_id", "inner") \
        # .groupBy("user_id").agg(F.collect_set())
        
    # print(topics_combined.count())

    topics_combined.select('*').filter(F.col("user_id") == '10f1bc81').show(100,truncate=False)
    
    
    pass


Questions we want to answer!

We can join on by users,
We can rank devices most likely to spend the most

In [94]:
run_etl(create_spark_session("ETL_job"),'../data/landing/transaction_events_1767986916.1293132.json', '../data/landing/user_events_1767986960.934074.json', "")

17
5
+--------+---------------------------------------------------------------+--------+------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+---------+--------+------+---------------------------+--------+------------------------------------+----------------+-------+----------------+-------+-------+----------+------------------------------------+----------------+--------------+--------------+----------+--------+------------+------------+---------------------------+
|user_id |billing_address                                                |currency|original_transaction_id             |payment_method|products                                                                                       